In [ ]:
!pip install tensorflow opencv-python matplotlib numpy

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import kagglehub
import os

# 1. Veri setini indir ve yolu al
path = kagglehub.dataset_download("asdasdasasdas/garbage-classification")
# İndirilen klasör genelde iç içe olabilir, ana klasörü bulalım
base_path = os.path.join(path, 'Garbage classification/Garbage classification')

# 2. Veri Hazırlama ve Çoğaltma (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, # %20'sini test için ayır
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 3. Model Oluşturma (Transfer Learning)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Hazır ağırlıkları dondur

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 4. Eğitim (Bilgisayarın gücüne göre epochs sayısını artırabilirsin)
print("Eğitim başlıyor...")
history = model.fit(
    train_generator,
    epochs=10, 
    validation_data=validation_generator
)

# 5. Modeli Kaydet (Sunumda kullanmak için çok önemli!)
model.save('atik_siniflandirma_modeli.h5')
print("Model başarıyla 'atik_siniflandirma_modeli.h5' adıyla kaydedildi.")